# Example Usage


## Start API Server

```bash
fastapi dev main.py
```

## Prepare Environment

In [18]:
import requests
from pathlib import Path
from boto_session_manager import BotoSesManager
from s3pathlib import S3Path, context
from unstructured_as_api.utils import bytes_to_str
from rich import print as rprint

dir_here = Path.cwd().absolute()
bsm = BotoSesManager(profile_name="bmt_app_dev_us_east_1")
context.attach_boto_session(bsm.boto_ses)

bucket = f"{bsm.aws_account_alias}-{bsm.aws_region}-data"
s3dir_root = S3Path(f"s3://{bucket}/projects/unstructured_as_api/")
s3dir_input = s3dir_root.joinpath("input/")
s3dir_output = s3dir_root.joinpath("output/")

s3path_in = s3dir_input.joinpath("f1040.pdf")
path = dir_here.joinpath("f1040.pdf")
s3path_in.write_bytes(path.read_bytes())

domain = "http://127.0.0.1:8000"

## Run partition PDF in sync

This is only suitable for small payload.

In [19]:
res = requests.post(
    f"{domain}/part_pdf/",
    json={
        "b64_content": bytes_to_str(path.read_bytes()),
    }
)
rprint(res.json()["elements"][:3])

[
    {
        'type': 'Header',
        'element_id': 'c9b88c54ac0941785ef13c4b9adee550',
        'text': 'm1040',
        'metadata': {
            'coordinates': {
                'points': [
                    [36.0960786, 35.65599999999995],
                    [36.0960786, 59.65599999999995],
                    [96.552, 59.65599999999995],
                    [96.552, 35.65599999999995]
                ],
                'system': 'PixelSpace',
                'layout_width': 612,
                'layout_height': 792
            },
            'languages': ['eng'],
            'page_number': 1,
            'filetype': 'application/pdf'
        }
    },
    {
        'type': 'Header',
        'element_id': '05b202019bdbc5a1627eab34b1fe38be',
        'text': 'r o F',
        'metadata': {
            'coordinates': {
                'points': [
                    [36.0960786, 44.55200000000002],
                    [36.0960786, 54.91899999999998],
                    [43.0959786, 54.91899999999998],
                    [43.0959786, 44.55200000000002]
                ],
                'system': 'PixelSpace',
                'layout_width': 612,
                'layout_height': 792
            },
            'languages': ['eng'],
            'page_number': 1,
            'filetype': 'application/pdf'
        }
    },
    {
        'type': 'Header',
        'element_id': 'ab963be57c2081e52af26aaeb279bc75',
        'text': 'Department of the Treasury—Internal Revenue Service U.S. Individual Income Tax Return',
        'metadata': {
            'coordinates': {
                'points': [
                    [97.6, 38.61799999999994],
                    [97.6, 59.230999999999995],
                    [283.8399999999999, 59.230999999999995],
                    [283.8399999999999, 38.61799999999994]
                ],
                'system': 'PixelSpace',
                'layout_width': 612,
                'layout_height': 792
            },
            'languages': ['eng'],
            'page_number': 1,
            'filetype': 'application/pdf'
        }
    }
]

## Run partition PDF using async API

Create an job and get job id.

In [20]:
res = requests.post(
    f"{domain}/start_part_pdf/",
    json={
        "s3uri_in": s3path_in.uri,
        "s3uri_out": s3dir_output.uri,
    }
)
res_data = res.json()
job_id = res_data["job_id"]
print(res_data)

{'job_id': 'a67ab5a7-74f9-40c1-874a-4c21080f9f80'}


Use job id to get the status and results.

In [21]:
res = requests.post(
    f"{domain}/get_part_pdf/",
    json={
        "job_id": job_id,
    }
)
res_data = res.json()
res_data["elements"] = res_data["elements"][:3]
rprint(res_data)

{
    'status': 'SUCCEEDED',
    'elements': [
        {
            'type': 'Header',
            'element_id': 'c9b88c54ac0941785ef13c4b9adee550',
            'text': 'm1040',
            'metadata': {
                'coordinates': {
                    'points': [
                        [36.0960786, 35.65599999999995],
                        [36.0960786, 59.65599999999995],
                        [96.552, 59.65599999999995],
                        [96.552, 35.65599999999995]
                    ],
                    'system': 'PixelSpace',
                    'layout_width': 612,
                    'layout_height': 792
                },
                'languages': ['eng'],
                'page_number': 1,
                'filetype': 'application/pdf'
            }
        },
        {
            'type': 'Header',
            'element_id': '05b202019bdbc5a1627eab34b1fe38be',
            'text': 'r o F',
            'metadata': {
                'coordinates': {
                    'points': [
                        [36.0960786, 44.55200000000002],
                        [36.0960786, 54.91899999999998],
                        [43.0959786, 54.91899999999998],
                        [43.0959786, 44.55200000000002]
                    ],
                    'system': 'PixelSpace',
                    'layout_width': 612,
                    'layout_height': 792
                },
                'languages': ['eng'],
                'page_number': 1,
                'filetype': 'application/pdf'
            }
        },
        {
            'type': 'Header',
            'element_id': 'ab963be57c2081e52af26aaeb279bc75',
            'text': 'Department of the Treasury—Internal Revenue Service U.S. Individual Income Tax Return',
            'metadata': {
                'coordinates': {
                    'points': [
                        [97.6, 38.61799999999994],
                        [97.6, 59.230999999999995],
                        [283.8399999999999, 59.230999999999995],
                        [283.8399999999999, 38.61799999999994]
                    ],
                    'system': 'PixelSpace',
                    'layout_width': 612,
                    'layout_height': 792
                },
                'languages': ['eng'],
                'page_number': 1,
                'filetype': 'application/pdf'
            }
        }
    ]
}